In [2]:
print(6)

6


In [1]:
import skfuzzy as fuzz

In [2]:
fuzz

<module 'skfuzzy' from '/home/garfield/anaconda3/lib/python3.8/site-packages/skfuzzy/__init__.py'>

In [3]:
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt


In [10]:
x_qual = np.arange(0, 11, 0.01)

In [11]:
x_qual

array([0.000e+00, 1.000e-02, 2.000e-02, ..., 1.097e+01, 1.098e+01,
       1.099e+01])

In [12]:
qual_lo = fuzz.trimf(x_qual, [0, 0, 5])

In [13]:
qual_lo 


array([1.   , 0.998, 0.996, ..., 0.   , 0.   , 0.   ])

In [14]:
fuzz.membership.gaussmf(x_qual, 0, 1)

array([1.00000000e+00, 9.99950001e-01, 9.99800020e-01, ...,
       7.38423639e-27, 6.61670499e-27, 5.92835938e-27])